In [ ]:
# Force reinstall all langchain packages to the latest matching versions
%pip install -U --force-reinstall langchain langchain-community langchain-core langchain-google-genai valyu prophet yfinance matplotlib pandas

  Using cached langchain-1.2.9-py3-none-any.whl.metadata (5.7 kB)
  Using cached langchain_community-0.4.1-py3-none-any.whl.metadata (3.0 kB)
  Using cached langchain_core-1.2.9-py3-none-any.whl.metadata (4.4 kB)
  Using cached langchain_google_genai-4.2.0-py3-none-any.whl.metadata (2.7 kB)
  Using cached valyu-2.5.4-py3-none-any.whl.metadata (22 kB)
  Using cached prophet-1.3.0-py3-none-macosx_11_0_arm64.whl.metadata (3.5 kB)
  Using cached yfinance-1.1.0-py2.py3-none-any.whl.metadata (6.1 kB)
  Using cached matplotlib-3.10.8-cp312-cp312-macosx_11_0_arm64.whl.metadata (52 kB)
  Using cached pandas-3.0.0-cp312-cp312-macosx_11_0_arm64.whl.metadata (79 kB)
  Using cached langgraph-1.0.8-py3-none-any.whl.metadata (7.4 kB)
  Using cached pydantic-2.12.5-py3-none-any.whl.metadata (90 kB)
  Using cached langchain_classic-1.0.1-py3-none-any.whl.metadata (4.2 kB)
  Using cached sqlalchemy-2.0.46-cp312-cp312-macosx_11_0_arm64.whl.metadata (9.5 kB)
  Using cached requests-2.32.5-py3-none-any.whl

In [ ]:
import os
import operator
import datetime
import numpy as np
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
from typing import Annotated, Literal, TypedDict, List

# --- LIBRARIES ---
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.agents import create_agent
from langchain_core.tools import tool
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langgraph.graph import StateGraph, START, END
from langgraph.types import Send
from pydantic import BaseModel, Field
from valyu import Valyu 





model = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=1.0,  
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # other params...
)


In [ ]:
@tool
def mlModel(ticker: str):
    """ 
    Uses Facebook Prophet to predict stock prices 30 days ahead.
    Returns daily price targets and saves a plot to a .png file.
    the png files name is generated by graph_filename = f"{ticker}_forecast.png"
    """

    try:
        # search stock data

        data = yf.Ticker(ticker).history(period="2y")
        if data.empty:
            return "Error: No data found"
        
        # Format for prophet
        
        df = data.reset_index()
        df['ds'] = df['Date'].dt.tz_localize(None)
        df['y'] = df['Close']

        # Train

        m = Prophet(daily_seasonality=True)
        m.fit(df)

        # Predict

        future = m.make_future_dataframe(periods=30)
        forecast = m.predict(future)

        # Save the Graph

        fig1 = m.plot(forecast)
        plt.title(f"{ticker} 30-Day Forecast")
        plt.xlabel("Date")
        plt.ylabel("Price")

        graph_filename = f"{ticker}_forecast.png"
        plt.savefig(graph_filename)
        plt.close()

        # List 30 day prediction
        future_data = forecast.tail(30)

        daily_tracking = []
        for index, row in future_data.iterrows():
            date_str = row['ds'].strftime('%Y-%m-%d')
            price_str = f"${row['yhat']:.2f}"
            daily_tracking.append(f"{date_str}: {price_str}")

        daily_summary = "\n".join(daily_tracking)

        latest_pred = forecast.iloc[-1]['yhat']
        current_price = df.iloc[-1]['y']
        trend = "UP" if latest_pred > current_price else "DOWN"
        
        # Return Everything
        return (f"Analysis Complete for {ticker}\n"
                f"Graph saved to: {graph_filename}\n"
                f"Trend: {trend}\n\n"
                f"Daily Price Targets (Next 30 Days):\n"
                f"{daily_summary}")
    
    except Exception as e:
        return f"Prediction failed: {e}"
    
    
    
@tool 
def brownianModel(ticker: str) -> str:
    """ 
    Uses geometric brownian motion and monte carlo method to predict stock prices 30 days ahead, for a given ticker. 
    Returns daily price predictions 30 trading days ahead, with a historical lookback period of 24 months.
    This can only be used if the stock has at least a two-year old history. 
    """
    
    # main variables
    scen_size = 10000
    HISTORICAL_YEARS = 2
    PREDICTION_DAYS = 30


    end_date = pd.Timestamp.today().normalize()
    start_date = end_date - pd.DateOffset(years=HISTORICAL_YEARS)
    pred_end_date = end_date + pd.tseries.offsets.BDay(PREDICTION_DAYS)


    # -----------------------------
    # Download and prepare data
    # -----------------------------

    prices = yf.download(tickers=ticker, start=start_date, end=end_date)

    if isinstance(prices.columns, pd.MultiIndex):
        prices = prices['Close'][ticker]
    else:
        prices = prices['Close']


    
    # Generate business days (weekdays only)
    future_dates = pd.bdate_range(start=end_date + pd.Timedelta(days=1), periods=PREDICTION_DAYS)


   
    train_set = prices.loc[:end_date]
    # Create DataFrame with a 'Date' column
    dframe = pd.DataFrame({'Prediction Date': future_dates})
    
    daily_returns = ((train_set / train_set.shift(1)) - 1)[1:]



    So = train_set.iloc[-1]
    N = PREDICTION_DAYS
    t = np.arange(1, N + 1)

    t = np.arange(1, N + 1)

    mu = np.mean(daily_returns)
    sigma = np.std(daily_returns)

    b = {str(scen): np.random.normal(0, 1, N) for scen in range(1, scen_size + 1)}
    W = {str(scen): b[str(scen)].cumsum() for scen in range(1, scen_size + 1)}

    drift = (mu - 0.5 * sigma ** 2) * t
    diffusion = {str(scen): sigma * W[str(scen)] for scen in range(1, scen_size + 1)}

    S = np.array([So * np.exp(drift + diffusion[str(scen)]) for scen in range(1, scen_size + 1)])
    S = np.hstack((np.array([[So] for _ in range(scen_size)]), S))


    S_max = [S[:, i].max() for i in range(0, N)]
    S_min = [S[:, i].min() for i in range(0, N)]
    S_pred = 0.5 * np.array(S_max) + 0.5 * np.array(S_min)


    # Standard Monte Carlo estimator: expected price (mean across simulations)


    # Align prediction length with available real prices
    #min_len = min(len(dframe['Prediction Date']), len(S_pred))


    # Generate 30 trading day future dates
    future_dates = pd.bdate_range(start=end_date + pd.Timedelta(days=1), periods=PREDICTION_DAYS)

    # Create DataFrame
    final_df = pd.DataFrame({
        'pred': S_pred
    }, index=future_dates)

    #mse = np.mean((final_df['pred'] - final_df['real']) ** 2)
    # Convert to string row by row as "Date: Price"

    result = '\n'.join(f"{date.date()}: {price}" for date, price in zip(final_df.index, final_df['pred']))
    return result



@tool
def valyu_search_tool(query: str):
    """
    Searches the web using Valyu API to get specific market news and context.
    """
    try:
        # Using the .answer method from your provided code
        response = valyu.answer(query)
        
        # Extract content safely
        if hasattr(response, 'contents'):
            return response.contents
        elif isinstance(response, dict) and 'contents' in response:
            return response['contents']
        else:
            return str(response)
            
    except Exception as e:
        return f"Search failed: {e}"



In [ ]:
trend_prompt = "You are a Quantitative Analyst. Use the provided ML and Statistical tools to analyze the stock ticker provided. Summarize the technical outlook."
trend_agent = create_agent(model, system_prompt=trend_prompt, tools=[mlModel, brownianModel])

noise_prompt = "You are a Market Researcher. Use the search tool to find recent news, sentiment, and macro factors affecting the stock."
noise_agent = create_agent(model, [valyu_search_tool], system_prompt=noise_prompt)


In [ ]:
from pydantic import BaseModel, Field
from langgraph.types import Send
from langgraph.graph import StateGraph, START, END



class AgentState(TypedDict):
    query: str
    results: Annotated[List[str], operator.add] 

def run_trend_agent(state: AgentState):
    """Invokes the Quant Agent"""
    print("Executing Trend Agent")
    response = trend_agent.invoke({"input": f"Analyze {state['query']}"})
    return {"results": [f"QUANT ANALYSIS:\n{response['output']}"]}

def run_noise_agent(state: AgentState):
    """Invokes the Research Agent"""
    print("Executing Noise Agent")
    response = noise_agent.invoke({"input": f"Find news for {state['query']}"})
    return {"results": [f"RESEARCH ANALYSIS:\n{response['output']}"]}

def aggregator(state: AgentState):
    """Combines results into a final answer"""
    print("Aggregating Results")
    final_prompt = (
        f"Combine the following reports into a comprehensive investment memo for {state['query']}:\n\n"
        + "\n\n".join(state["results"])
    )
    response = model.invoke(final_prompt)
    print(f"\nFINAL ANSWER:\n{response.content}")
    return {"results": [response.content]}

class RouteSchema(BaseModel):
    targets: List[Literal["quant", "research"]] = Field(
        description="Which agents to hire? Quant for numbers/charts, Research for news/sentiment."
    )

workflow = StateGraph(AgentState)


workflow.add_node("trend_node", run_trend_agent)
workflow.add_node("noise_node", run_noise_agent)
workflow.add_node("aggregator", aggregator)

def route_query(state: AgentState) -> List[Send]:
    structured_llm = model.with_structured_output(RouteSchema)
    decision = structured_llm.invoke(f"Analyze: {state['query']}")
    
    routes = []
    if "quant" in decision.targets:
        routes.append(Send("trend_node", state))
    if "research" in decision.targets:
        routes.append(Send("noise_node", state))
    if not routes:
        routes = [Send("trend_node", state), Send("noise_node", state)]
        
    return routes

workflow.add_conditional_edges(START, route_query)
workflow.add_edge("trend_node", "aggregator")
workflow.add_edge("noise_node", "aggregator")
workflow.add_edge("aggregator", END)

app = workflow.compile()



if __name__ == "__main__":
    user_query = "What is the outlook for AAPL stock?"
    inputs = {"query": user_query, "results": []}

    for output in app.stream(inputs):
        pass 









Failed to send compressed multipart ingest: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')


Executing Trend AgentExecuting Noise Agent



ValidationError: 4 validation errors for Schema
properties.TICKER
  Input should be a valid dictionary or object to extract fields from [type=model_attributes_type, input_value=None, input_type=NoneType]
    For further information visit https://errors.pydantic.dev/2.12/v/model_attributes_type
properties.START_DATE
  Input should be a valid dictionary or object to extract fields from [type=model_attributes_type, input_value=None, input_type=NoneType]
    For further information visit https://errors.pydantic.dev/2.12/v/model_attributes_type
properties.END_DATE
  Input should be a valid dictionary or object to extract fields from [type=model_attributes_type, input_value=None, input_type=NoneType]
    For further information visit https://errors.pydantic.dev/2.12/v/model_attributes_type
properties.PRED_END_DATE
  Input should be a valid dictionary or object to extract fields from [type=model_attributes_type, input_value=None, input_type=NoneType]
    For further information visit https://

Failed to send compressed multipart ingest: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')
